In [46]:
import os
import vertexai
from dotenv import load_dotenv

from sqlalchemy import (
     create_engine,
     MetaData,
     Table,
     Column,
     String,
     Integer,
     select,
 )

from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

from llama_index.core import Settings
from llama_index.core import SQLDatabase
from llama_index.llms.google_genai import GoogleGenAI
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

In [27]:
load_dotenv("../../agents/.env")

True

In [28]:
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
DATASET_ID = "sandbox_rsolar"

In [29]:
vertexai.init(project=os.getenv("GOOGLE_CLOUD_PROJECT"), location=os.getenv("GOOGLE_CLOUD_LOCATION"))

In [30]:
#initiate bigquery connection
dataset_uri = f"bigquery://{PROJECT_ID}/{DATASET_ID}"
bq_engine = create_engine(dataset_uri)

#initiate database connection
sql_database = SQLDatabase(bq_engine)

In [43]:
llm = GoogleGenAI(model="gemini-2.0-flash-001",
                  max_tokens=8192,
                  temperature=0.1,  additional_kwargs={})

embed_model = LangchainEmbedding(VertexAIEmbeddings(model_name='text-embedding-004'))

Settings.llm = llm
Settings.embed_model = embed_model

In [47]:
# add contextual description in the schema object
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="estandar_aa",
                    context_str="La tabla estandar_aa constituye un detallado catálogo de estándares y buenas prácticas, diseñado para la evaluación y certificación en el sector agroindustrial. Cada entrada, identificada por un n numérico y un codigo alfanumérico único, se clasifica por su nivel de exigencia (como Fundamental o Avanzado) y se le asigna una puntos específica. Estos estándares se agrupan por dimension (Ambiente, Calidad, etc.) y tema (Agua, Biodiversidad), describiendo una buena_practica general, la accion concreta que la planta debe implementar, el medio_de_verificacion necesario para demostrar su cumplimiento, y un link_recursos para información adicional.") )
]

In [48]:
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex
)

In [49]:
bq_retriever = SQLTableRetrieverQueryEngine(
     sql_database,
     obj_index.as_retriever(similarity_top_k=2),
     embed_model=embed_model,
     llm=llm,
     synthesize_response=True,
 )

In [55]:
query = "Dame la lista de enlaces asociados a la tema agua"
response = bq_retriever.query(query)
display(Markdown(f"{response}"))

Los enlaces asociados al tema agua son:

*   https://ciruelacertificada.cl/?s=A002&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A003&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A004&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A007&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A008&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A009&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A014&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A010&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A011&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A012&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A015&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A016&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A005&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A006&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A013&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A017&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...
*   https://ciruelacertificada.cl/?s=A001&jet\_ajax\_search\_settings=%7B%22current\_query%22%3A%7B%22taxonomy%22%3A%22estandares-recursos%22%2C%22term%22%3A%22recurso-adecuacion-agroindustrial%22%7D%2C%22sentence%22%3Atrue%2C%22search\_in\_taxonomy%22%3Atrue%2C%22search\_in\_taxonomy\_source%22%3A%5B%22eleme...


In [ ]:
response.metadata['sql_query']

"SELECT\n    estandar_aa.link_recursos\nFROM\n    estandar_aa\nWHERE\n    estandar_aa.tema = 'Agua'"